In [1]:
push!(LOAD_PATH, pwd())
using DHC_2DUtils
using Profile
using BenchmarkTools
using FFTW
using HDF5
using Test
using SparseArrays
using Statistics
using Plots
using LinearAlgebra
using Measures
using Plots
using MLDatasets
using ImageCore
using Colors
using AbstractFFTs
using Interpolations
using DSP
using StatsBase
using Images

  likely near /n/home12/saydjari/.julia/packages/Plots/uCh2y/src/backends/hdf5.jl:162
  likely near /n/home12/saydjari/.julia/packages/Plots/uCh2y/src/backends/hdf5.jl:162
  likely near /n/home12/saydjari/.julia/packages/Plots/uCh2y/src/backends/hdf5.jl:166
  likely near /n/home12/saydjari/.julia/packages/Plots/uCh2y/src/backends/hdf5.jl:166
  likely near /n/home12/saydjari/.julia/packages/Plots/uCh2y/src/backends/hdf5.jl:176
  likely near /n/home12/saydjari/.julia/packages/Plots/uCh2y/src/backends/hdf5.jl:179
  likely near /n/home12/saydjari/.julia/packages/Plots/uCh2y/src/backends/hdf5.jl:181
  likely near /n/home12/saydjari/.julia/packages/Plots/uCh2y/src/backends/hdf5.jl:184
  likely near /n/home12/saydjari/.julia/packages/Plots/uCh2y/src/backends/hdf5.jl:186
  likely near /n/home12/saydjari/.julia/packages/Plots/uCh2y/src/backends/hdf5.jl:189
  likely near /n/home12/saydjari/.julia/packages/Plots/uCh2y/src/backends/hdf5.jl:203
  likely near /n/home12/saydjari/.julia/packages/Plots

In [2]:
using Distributed
using ProgressMeter

In [3]:
train_x, train_y = CIFAR100.traindata();
test_x, test_y   = CIFAR100.testdata();

In [4]:
function color_convert(array, color1, color2)
    chan_img = channelview(array)
    cv = colorview(color1, StackedView(array[:,:,1],array[:,:,2],array[:,:,3]))
    y_im = color2.(cv)
    channels = permutedims(channelview(float.(y_im)),(2,3,1))
    return channels
end

color_convert (generic function with 1 method)

In [5]:
lst_train = Array{Any}(undef, 0)
for i = 1:size(train_x)[4]
    push!(lst_train,train_x[:,:,:,i])
end

lst_test = Array{Any}(undef, 0)
for i = 1:size(test_x)[4]
    push!(lst_test,test_x[:,:,:,i])
end

In [5]:
addprocs(30)

30-element Array{Int64,1}:
  2
  3
  4
  5
  6
  7
  8
  9
 10
 11
 12
 13
 14
  ⋮
 20
 21
 22
 23
 24
 25
 26
 27
 28
 29
 30
 31

In [6]:
@everywhere begin    
    push!(LOAD_PATH, pwd())
    using DHC_2DUtils
    using Profile
    using BenchmarkTools
    using FFTW
    using Test
    using SparseArrays
    using Statistics
    using Plots
    using LinearAlgebra
    using Measures
    using Plots
    using MLDatasets
    using ImageCore
    using Colors
    using AbstractFFTs
    using Interpolations
    using DSP
    using StatsBase
    using Distributed
    using Images
    using ProgressMeter
    
    # filter bank
    filter_hash = fink_filter_hash(1, 8, nx=128, pc=1, wd=2)

    function wind_2d(nx)
        dx   = nx/2-1
        filter = zeros(Float64, nx, nx)
        A = DSP.tukey(nx, 0.3)
        itp = extrapolate(interpolate(A,BSpline(Linear())),0)
        @inbounds for x = 1:nx
            sx = x-dx-1    # define sx,sy so that no fftshift() needed
            for y = 1:nx
                sy = y-dx-1
                r  = sqrt.((sx).^2 + (sy).^2) + nx/2
                filter[x,y] = itp(r)
            end
        end
        return filter
    end

    function wind_2d_RGB(nx)
        dx   = nx/2-1
        filter = zeros(Float64, nx, nx)
        A = DSP.tukey(nx, 0.3)
        itp = extrapolate(interpolate(A,BSpline(Linear())),0)
        @inbounds for x = 1:nx
            sx = x-dx-1    # define sx,sy so that no fftshift() needed
            for y = 1:nx
                sy = y-dx-1
                r  = sqrt.((sx).^2 + (sy).^2) + nx/2
                filter[x,y] = itp(r)
            end
        end
        return reshape(filter,nx,nx,1)
    end

    function cifar_pad_RGB(im; θ=0.0)
        imbig = convert(Array{Float64,3},imresize(im,(64,64,3)))
        datad_w = fweights(wind_2d(64));
        mu_imbig = zeros(1,1,3)
        for chan = 1:3
            mu_imbig[chan] = mean(imbig[:,:,chan],datad_w)
        end
        imbig .-= mu_imbig
        imbig .*= wind_2d_RGB(64)
        impad = zeros(Float64,128,128,3)
        impad[96:-1:33,33:96,:] = imbig

        if θ != 0.0
            imrot = imrotate(impad, θ, axes(impad), Cubic(Throw(OnGrid())))
            imrot[findall(imrot .!= imrot)] .= 0.0
            return imrot .+ mu_imbig
        end

        return impad.+ mu_imbig
    end

    function cifar_DHC(x)
        image = cifar_pad_RGB(x[:,:,:], θ=0.0)
        WST = DHC_compute_RGB(image, filter_hash)
        return WST
    end
end

      From worker 22:	WARNING: importing deprecated binding HDF5.HDF5Group into _hdf5_implementation.
      From worker 22:	WARNING: importing deprecated binding HDF5.HDF5Dataset into _hdf5_implementation.
      From worker 11:	WARNING: importing deprecated binding HDF5.HDF5Group into _hdf5_implementation.
      From worker 11:	WARNING: importing deprecated binding HDF5.HDF5Dataset into _hdf5_implementation.
      From worker 19:	WARNING: importing deprecated binding HDF5.HDF5Group into _hdf5_implementation.
      From worker 19:	WARNING: importing deprecated binding HDF5.HDF5Dataset into _hdf5_implementation.
      From worker 10:	WARNING: importing deprecated binding HDF5.HDF5Group into _hdf5_implementation.
      From worker 10:	WARNING: importing deprecated binding HDF5.HDF5Dataset into _hdf5_implementation.
      From worker 3:	WARNING: importing deprecated binding HDF5.HDF5Group into _hdf5_implementation.
      From worker 3:	WARNING: importing deprecated binding HDF5.HDF5Dataset

In [8]:
h5write("cifar100_RGB.h5", "main/test_labels", test_y, deflate=3)
h5write("cifar100_RGB.h5", "main/train_labels", train_y, deflate=3)

cifar_DHC_out = @showprogress pmap(cifar_DHC, lst_train)
cifar_DHC_out = hcat(cifar_DHC_out...)

h5write("cifar100_RGB.h5", "main/train_data", cifar_DHC_out, deflate=3)

cifar_DHC_out_iso = transformMaker(cifar_DHC_out',filter_hash["S1_iso_mat"],filter_hash["S2_iso_mat"],Nc=3)

h5write("cifar100_RGB.h5", "main/train_data_iso", cifar_DHC_out_iso, deflate=3)

cifar_DHC_out = @showprogress pmap(cifar_DHC, lst_test)
cifar_DHC_out = hcat(cifar_DHC_out...)

h5write("cifar100_RGB.h5", "main/test_data", cifar_DHC_out, deflate=3)

cifar_DHC_out_iso = transformMaker(cifar_DHC_out',filter_hash["S1_iso_mat"],filter_hash["S2_iso_mat"],Nc=3)

h5write("cifar100_RGB.h5", "main/test_data_iso", cifar_DHC_out_iso, deflate=3)

Progress: 100%|█████████████████████████████████████████| Time: 0:09:12
Progress: 100%|█████████████████████████████████████████| Time: 0:01:01


In [9]:
train_x_ycbcr = zeros(size(train_x))
for i=1:size(train_x)[4]
    train_x_ycbcr[:,:,:,i] = color_convert(train_x[:,:,:,i],RGB,YCbCr)
end
test_x_ycbcr = zeros(size(test_x))
for i=1:size(test_x)[4]
    test_x_ycbcr[:,:,:,i] = color_convert(test_x[:,:,:,i],RGB,YCbCr)
end

lst_train = Array{Any}(undef, 0)
for i = 1:size(train_x_ycbcr)[4]
    push!(lst_train,train_x_ycbcr[:,:,:,i])
end

lst_test = Array{Any}(undef, 0)
for i = 1:size(test_x_ycbcr)[4]
    push!(lst_test,test_x_ycbcr[:,:,:,i])
end

In [10]:
h5write("cifar100_YCbCr.h5", "main/test_labels", test_y, deflate=3)
h5write("cifar100_YCbCr.h5", "main/train_labels", train_y, deflate=3)

cifar_DHC_out = @showprogress pmap(cifar_DHC, lst_train)
cifar_DHC_out = hcat(cifar_DHC_out...)

h5write("cifar100_YCbCr.h5", "main/train_data", cifar_DHC_out, deflate=3)

cifar_DHC_out_iso = transformMaker(cifar_DHC_out',filter_hash["S1_iso_mat"],filter_hash["S2_iso_mat"],Nc=3)

h5write("cifar100_YCbCr.h5", "main/train_data_iso", cifar_DHC_out_iso, deflate=3)

cifar_DHC_out = @showprogress pmap(cifar_DHC, lst_test)
cifar_DHC_out = hcat(cifar_DHC_out...)

h5write("cifar100_YCbCr.h5", "main/test_data", cifar_DHC_out, deflate=3)

cifar_DHC_out_iso = transformMaker(cifar_DHC_out',filter_hash["S1_iso_mat"],filter_hash["S2_iso_mat"],Nc=3)

h5write("cifar100_YCbCr.h5", "main/test_data_iso", cifar_DHC_out_iso, deflate=3)

Progress: 100%|█████████████████████████████████████████| Time: 0:08:45
Progress: 100%|█████████████████████████████████████████| Time: 0:01:02


In [11]:
train_x_ycbcr = zeros(size(train_x))
for i=1:size(train_x)[4]
    train_x_ycbcr[:,:,:,i] = color_convert(train_x[:,:,:,i],RGB,YIQ)
end
test_x_ycbcr = zeros(size(test_x))
for i=1:size(test_x)[4]
    test_x_ycbcr[:,:,:,i] = color_convert(test_x[:,:,:,i],RGB,YIQ)
end

lst_train = Array{Any}(undef, 0)
for i = 1:size(train_x_ycbcr)[4]
    push!(lst_train,train_x_ycbcr[:,:,:,i])
end

lst_test = Array{Any}(undef, 0)
for i = 1:size(test_x_ycbcr)[4]
    push!(lst_test,test_x_ycbcr[:,:,:,i])
end

In [12]:
filename = "cifar100_YIQ.h5"

h5write(filename, "main/test_labels", test_y, deflate=3)
h5write(filename, "main/train_labels", train_y, deflate=3)

cifar_DHC_out = @showprogress pmap(cifar_DHC, lst_train)
cifar_DHC_out = hcat(cifar_DHC_out...)

h5write(filename, "main/train_data", cifar_DHC_out, deflate=3)

cifar_DHC_out_iso = transformMaker(cifar_DHC_out',filter_hash["S1_iso_mat"],filter_hash["S2_iso_mat"],Nc=3)

h5write(filename, "main/train_data_iso", cifar_DHC_out_iso, deflate=3)

cifar_DHC_out = @showprogress pmap(cifar_DHC, lst_test)
cifar_DHC_out = hcat(cifar_DHC_out...)

h5write(filename, "main/test_data", cifar_DHC_out, deflate=3)

cifar_DHC_out_iso = transformMaker(cifar_DHC_out',filter_hash["S1_iso_mat"],filter_hash["S2_iso_mat"],Nc=3)

h5write(filename, "main/test_data_iso", cifar_DHC_out_iso, deflate=3)

Progress: 100%|█████████████████████████████████████████| Time: 0:08:10
Progress: 100%|█████████████████████████████████████████| Time: 0:01:01


In [13]:
train_x_ycbcr = zeros(size(train_x))
for i=1:size(train_x)[4]
    train_x_ycbcr[:,:,:,i] = color_convert(train_x[:,:,:,i],RGB,HSV)
end
test_x_ycbcr = zeros(size(test_x))
for i=1:size(test_x)[4]
    test_x_ycbcr[:,:,:,i] = color_convert(test_x[:,:,:,i],RGB,HSV)
end

lst_train = Array{Any}(undef, 0)
for i = 1:size(train_x_ycbcr)[4]
    push!(lst_train,train_x_ycbcr[:,:,:,i])
end

lst_test = Array{Any}(undef, 0)
for i = 1:size(test_x_ycbcr)[4]
    push!(lst_test,test_x_ycbcr[:,:,:,i])
end

In [14]:
filename = "cifar100_HSV.h5"

h5write(filename, "main/test_labels", test_y, deflate=3)
h5write(filename, "main/train_labels", train_y, deflate=3)

cifar_DHC_out = @showprogress pmap(cifar_DHC, lst_train)
cifar_DHC_out = hcat(cifar_DHC_out...)

h5write(filename, "main/train_data", cifar_DHC_out, deflate=3)

cifar_DHC_out_iso = transformMaker(cifar_DHC_out',filter_hash["S1_iso_mat"],filter_hash["S2_iso_mat"],Nc=3)

h5write(filename, "main/train_data_iso", cifar_DHC_out_iso, deflate=3)

cifar_DHC_out = @showprogress pmap(cifar_DHC, lst_test)
cifar_DHC_out = hcat(cifar_DHC_out...)

h5write(filename, "main/test_data", cifar_DHC_out, deflate=3)

cifar_DHC_out_iso = transformMaker(cifar_DHC_out',filter_hash["S1_iso_mat"],filter_hash["S2_iso_mat"],Nc=3)

h5write(filename, "main/test_data_iso", cifar_DHC_out_iso, deflate=3)

Progress: 100%|█████████████████████████████████████████| Time: 0:08:15
Progress: 100%|█████████████████████████████████████████| Time: 0:03:20


In [15]:
train_x_ycbcr = zeros(size(train_x))
for i=1:size(train_x)[4]
    train_x_ycbcr[:,:,:,i] = color_convert(train_x[:,:,:,i],RGB,HSL)
end
test_x_ycbcr = zeros(size(test_x))
for i=1:size(test_x)[4]
    test_x_ycbcr[:,:,:,i] = color_convert(test_x[:,:,:,i],RGB,HSL)
end

lst_train = Array{Any}(undef, 0)
for i = 1:size(train_x_ycbcr)[4]
    push!(lst_train,train_x_ycbcr[:,:,:,i])
end

lst_test = Array{Any}(undef, 0)
for i = 1:size(test_x_ycbcr)[4]
    push!(lst_test,test_x_ycbcr[:,:,:,i])
end

In [16]:
filename = "cifar100_HSL.h5"

h5write(filename, "main/test_labels", test_y, deflate=3)
h5write(filename, "main/train_labels", train_y, deflate=3)

cifar_DHC_out = @showprogress pmap(cifar_DHC, lst_train)
cifar_DHC_out = hcat(cifar_DHC_out...)

h5write(filename, "main/train_data", cifar_DHC_out, deflate=3)

cifar_DHC_out_iso = transformMaker(cifar_DHC_out',filter_hash["S1_iso_mat"],filter_hash["S2_iso_mat"],Nc=3)

h5write(filename, "main/train_data_iso", cifar_DHC_out_iso, deflate=3)

cifar_DHC_out = @showprogress pmap(cifar_DHC, lst_test)
cifar_DHC_out = hcat(cifar_DHC_out...)

h5write(filename, "main/test_data", cifar_DHC_out, deflate=3)

cifar_DHC_out_iso = transformMaker(cifar_DHC_out',filter_hash["S1_iso_mat"],filter_hash["S2_iso_mat"],Nc=3)

h5write(filename, "main/test_data_iso", cifar_DHC_out_iso, deflate=3)

Progress: 100%|█████████████████████████████████████████| Time: 0:08:35
Progress: 100%|█████████████████████████████████████████| Time: 0:03:26


In [17]:
train_x_ycbcr = zeros(size(train_x))
for i=1:size(train_x)[4]
    train_x_ycbcr[:,:,:,i] = color_convert(train_x[:,:,:,i],RGB,HSI)
end
test_x_ycbcr = zeros(size(test_x))
for i=1:size(test_x)[4]
    test_x_ycbcr[:,:,:,i] = color_convert(test_x[:,:,:,i],RGB,HSI)
end

lst_train = Array{Any}(undef, 0)
for i = 1:size(train_x_ycbcr)[4]
    push!(lst_train,train_x_ycbcr[:,:,:,i])
end

lst_test = Array{Any}(undef, 0)
for i = 1:size(test_x_ycbcr)[4]
    push!(lst_test,test_x_ycbcr[:,:,:,i])
end

In [18]:
filename = "cifar100_HSI.h5"

h5write(filename, "main/test_labels", test_y, deflate=3)
h5write(filename, "main/train_labels", train_y, deflate=3)

cifar_DHC_out = @showprogress pmap(cifar_DHC, lst_train)
cifar_DHC_out = hcat(cifar_DHC_out...)

h5write(filename, "main/train_data", cifar_DHC_out, deflate=3)

cifar_DHC_out_iso = transformMaker(cifar_DHC_out',filter_hash["S1_iso_mat"],filter_hash["S2_iso_mat"],Nc=3)

h5write(filename, "main/train_data_iso", cifar_DHC_out_iso, deflate=3)

cifar_DHC_out = @showprogress pmap(cifar_DHC, lst_test)
cifar_DHC_out = hcat(cifar_DHC_out...)

h5write(filename, "main/test_data", cifar_DHC_out, deflate=3)

cifar_DHC_out_iso = transformMaker(cifar_DHC_out',filter_hash["S1_iso_mat"],filter_hash["S2_iso_mat"],Nc=3)

h5write(filename, "main/test_data_iso", cifar_DHC_out_iso, deflate=3)

Progress: 100%|█████████████████████████████████████████| Time: 0:08:30
Progress: 100%|█████████████████████████████████████████| Time: 0:03:17


In [19]:
train_x_ycbcr = zeros(size(train_x))
for i=1:size(train_x)[4]
    train_x_ycbcr[:,:,:,i] = color_convert(train_x[:,:,:,i],RGB,Lab)
end
test_x_ycbcr = zeros(size(test_x))
for i=1:size(test_x)[4]
    test_x_ycbcr[:,:,:,i] = color_convert(test_x[:,:,:,i],RGB,Lab)
end

lst_train = Array{Any}(undef, 0)
for i = 1:size(train_x_ycbcr)[4]
    push!(lst_train,train_x_ycbcr[:,:,:,i])
end

lst_test = Array{Any}(undef, 0)
for i = 1:size(test_x_ycbcr)[4]
    push!(lst_test,test_x_ycbcr[:,:,:,i])
end

In [20]:
filename = "cifar100_Lab.h5"

h5write(filename, "main/test_labels", test_y, deflate=3)
h5write(filename, "main/train_labels", train_y, deflate=3)

cifar_DHC_out = @showprogress pmap(cifar_DHC, lst_train)
cifar_DHC_out = hcat(cifar_DHC_out...)

h5write(filename, "main/train_data", cifar_DHC_out, deflate=3)

cifar_DHC_out_iso = transformMaker(cifar_DHC_out',filter_hash["S1_iso_mat"],filter_hash["S2_iso_mat"],Nc=3)

h5write(filename, "main/train_data_iso", cifar_DHC_out_iso, deflate=3)

cifar_DHC_out = @showprogress pmap(cifar_DHC, lst_test)
cifar_DHC_out = hcat(cifar_DHC_out...)

h5write(filename, "main/test_data", cifar_DHC_out, deflate=3)

cifar_DHC_out_iso = transformMaker(cifar_DHC_out',filter_hash["S1_iso_mat"],filter_hash["S2_iso_mat"],Nc=3)

h5write(filename, "main/test_data_iso", cifar_DHC_out_iso, deflate=3)

Progress: 100%|█████████████████████████████████████████| Time: 0:08:31
Progress: 100%|█████████████████████████████████████████| Time: 0:03:20


In [21]:
train_x_ycbcr = zeros(size(train_x))
for i=1:size(train_x)[4]
    train_x_ycbcr[:,:,:,i] = color_convert(train_x[:,:,:,i],RGB,Luv)
end
test_x_ycbcr = zeros(size(test_x))
for i=1:size(test_x)[4]
    test_x_ycbcr[:,:,:,i] = color_convert(test_x[:,:,:,i],RGB,Luv)
end

lst_train = Array{Any}(undef, 0)
for i = 1:size(train_x_ycbcr)[4]
    push!(lst_train,train_x_ycbcr[:,:,:,i])
end

lst_test = Array{Any}(undef, 0)
for i = 1:size(test_x_ycbcr)[4]
    push!(lst_test,test_x_ycbcr[:,:,:,i])
end

In [22]:
filename = "cifar100_Luv.h5"

h5write(filename, "main/test_labels", test_y, deflate=3)
h5write(filename, "main/train_labels", train_y, deflate=3)

cifar_DHC_out = @showprogress pmap(cifar_DHC, lst_train)
cifar_DHC_out = hcat(cifar_DHC_out...)

h5write(filename, "main/train_data", cifar_DHC_out, deflate=3)

cifar_DHC_out_iso = transformMaker(cifar_DHC_out',filter_hash["S1_iso_mat"],filter_hash["S2_iso_mat"],Nc=3)

h5write(filename, "main/train_data_iso", cifar_DHC_out_iso, deflate=3)

cifar_DHC_out = @showprogress pmap(cifar_DHC, lst_test)
cifar_DHC_out = hcat(cifar_DHC_out...)

h5write(filename, "main/test_data", cifar_DHC_out, deflate=3)

cifar_DHC_out_iso = transformMaker(cifar_DHC_out',filter_hash["S1_iso_mat"],filter_hash["S2_iso_mat"],Nc=3)

h5write(filename, "main/test_data_iso", cifar_DHC_out_iso, deflate=3)

Progress: 100%|█████████████████████████████████████████| Time: 0:08:26
Progress: 100%|█████████████████████████████████████████| Time: 0:03:14


In [23]:
train_x_ycbcr = zeros(size(train_x))
for i=1:size(train_x)[4]
    train_x_ycbcr[:,:,:,i] = color_convert(train_x[:,:,:,i],RGB,LCHab)
end
test_x_ycbcr = zeros(size(test_x))
for i=1:size(test_x)[4]
    test_x_ycbcr[:,:,:,i] = color_convert(test_x[:,:,:,i],RGB,LCHab)
end

lst_train = Array{Any}(undef, 0)
for i = 1:size(train_x_ycbcr)[4]
    push!(lst_train,train_x_ycbcr[:,:,:,i])
end

lst_test = Array{Any}(undef, 0)
for i = 1:size(test_x_ycbcr)[4]
    push!(lst_test,test_x_ycbcr[:,:,:,i])
end

filename = "cifar100_LCHab.h5"

h5write(filename, "main/test_labels", test_y, deflate=3)
h5write(filename, "main/train_labels", train_y, deflate=3)

cifar_DHC_out = @showprogress pmap(cifar_DHC, lst_train)
cifar_DHC_out = hcat(cifar_DHC_out...)

h5write(filename, "main/train_data", cifar_DHC_out, deflate=3)

cifar_DHC_out_iso = transformMaker(cifar_DHC_out',filter_hash["S1_iso_mat"],filter_hash["S2_iso_mat"],Nc=3)

h5write(filename, "main/train_data_iso", cifar_DHC_out_iso, deflate=3)

cifar_DHC_out = @showprogress pmap(cifar_DHC, lst_test)
cifar_DHC_out = hcat(cifar_DHC_out...)

h5write(filename, "main/test_data", cifar_DHC_out, deflate=3)

cifar_DHC_out_iso = transformMaker(cifar_DHC_out',filter_hash["S1_iso_mat"],filter_hash["S2_iso_mat"],Nc=3)

h5write(filename, "main/test_data_iso", cifar_DHC_out_iso, deflate=3)

Progress: 100%|█████████████████████████████████████████| Time: 0:08:37
Progress: 100%|█████████████████████████████████████████| Time: 0:03:19


In [24]:
train_x_ycbcr = zeros(size(train_x))
for i=1:size(train_x)[4]
    train_x_ycbcr[:,:,:,i] = color_convert(train_x[:,:,:,i],RGB,LCHuv)
end
test_x_ycbcr = zeros(size(test_x))
for i=1:size(test_x)[4]
    test_x_ycbcr[:,:,:,i] = color_convert(test_x[:,:,:,i],RGB,LCHuv)
end

lst_train = Array{Any}(undef, 0)
for i = 1:size(train_x_ycbcr)[4]
    push!(lst_train,train_x_ycbcr[:,:,:,i])
end

lst_test = Array{Any}(undef, 0)
for i = 1:size(test_x_ycbcr)[4]
    push!(lst_test,test_x_ycbcr[:,:,:,i])
end

filename = "cifar100_LCHuv.h5"

h5write(filename, "main/test_labels", test_y, deflate=3)
h5write(filename, "main/train_labels", train_y, deflate=3)

cifar_DHC_out = @showprogress pmap(cifar_DHC, lst_train)
cifar_DHC_out = hcat(cifar_DHC_out...)

h5write(filename, "main/train_data", cifar_DHC_out, deflate=3)

cifar_DHC_out_iso = transformMaker(cifar_DHC_out',filter_hash["S1_iso_mat"],filter_hash["S2_iso_mat"],Nc=3)

h5write(filename, "main/train_data_iso", cifar_DHC_out_iso, deflate=3)

cifar_DHC_out = @showprogress pmap(cifar_DHC, lst_test)
cifar_DHC_out = hcat(cifar_DHC_out...)

h5write(filename, "main/test_data", cifar_DHC_out, deflate=3)

cifar_DHC_out_iso = transformMaker(cifar_DHC_out',filter_hash["S1_iso_mat"],filter_hash["S2_iso_mat"],Nc=3)

h5write(filename, "main/test_data_iso", cifar_DHC_out_iso, deflate=3)

Progress: 100%|█████████████████████████████████████████| Time: 0:08:23
Progress: 100%|█████████████████████████████████████████| Time: 0:03:20


In [25]:
train_x_ycbcr = zeros(size(train_x))
for i=1:size(train_x)[4]
    train_x_ycbcr[:,:,:,i] = color_convert(train_x[:,:,:,i],RGB,XYZ)
end
test_x_ycbcr = zeros(size(test_x))
for i=1:size(test_x)[4]
    test_x_ycbcr[:,:,:,i] = color_convert(test_x[:,:,:,i],RGB,XYZ)
end

lst_train = Array{Any}(undef, 0)
for i = 1:size(train_x_ycbcr)[4]
    push!(lst_train,train_x_ycbcr[:,:,:,i])
end

lst_test = Array{Any}(undef, 0)
for i = 1:size(test_x_ycbcr)[4]
    push!(lst_test,test_x_ycbcr[:,:,:,i])
end

filename = "cifar100_XYZ.h5"

h5write(filename, "main/test_labels", test_y, deflate=3)
h5write(filename, "main/train_labels", train_y, deflate=3)

cifar_DHC_out = @showprogress pmap(cifar_DHC, lst_train)
cifar_DHC_out = hcat(cifar_DHC_out...)

h5write(filename, "main/train_data", cifar_DHC_out, deflate=3)

cifar_DHC_out_iso = transformMaker(cifar_DHC_out',filter_hash["S1_iso_mat"],filter_hash["S2_iso_mat"],Nc=3)

h5write(filename, "main/train_data_iso", cifar_DHC_out_iso, deflate=3)

cifar_DHC_out = @showprogress pmap(cifar_DHC, lst_test)
cifar_DHC_out = hcat(cifar_DHC_out...)

h5write(filename, "main/test_data", cifar_DHC_out, deflate=3)

cifar_DHC_out_iso = transformMaker(cifar_DHC_out',filter_hash["S1_iso_mat"],filter_hash["S2_iso_mat"],Nc=3)

h5write(filename, "main/test_data_iso", cifar_DHC_out_iso, deflate=3)

Progress: 100%|█████████████████████████████████████████| Time: 0:08:15
Progress: 100%|█████████████████████████████████████████| Time: 0:03:18


In [26]:
train_x_ycbcr = zeros(size(train_x))
for i=1:size(train_x)[4]
    train_x_ycbcr[:,:,:,i] = color_convert(train_x[:,:,:,i],RGB,xyY)
end
test_x_ycbcr = zeros(size(test_x))
for i=1:size(test_x)[4]
    test_x_ycbcr[:,:,:,i] = color_convert(test_x[:,:,:,i],RGB,xyY)
end

lst_train = Array{Any}(undef, 0)
for i = 1:size(train_x_ycbcr)[4]
    push!(lst_train,train_x_ycbcr[:,:,:,i])
end

lst_test = Array{Any}(undef, 0)
for i = 1:size(test_x_ycbcr)[4]
    push!(lst_test,test_x_ycbcr[:,:,:,i])
end

filename = "cifar100_xyY.h5"

h5write(filename, "main/test_labels", test_y, deflate=3)
h5write(filename, "main/train_labels", train_y, deflate=3)

cifar_DHC_out = @showprogress pmap(cifar_DHC, lst_train)
cifar_DHC_out = hcat(cifar_DHC_out...)

h5write(filename, "main/train_data", cifar_DHC_out, deflate=3)

cifar_DHC_out_iso = transformMaker(cifar_DHC_out',filter_hash["S1_iso_mat"],filter_hash["S2_iso_mat"],Nc=3)

h5write(filename, "main/train_data_iso", cifar_DHC_out_iso, deflate=3)

cifar_DHC_out = @showprogress pmap(cifar_DHC, lst_test)
cifar_DHC_out = hcat(cifar_DHC_out...)

h5write(filename, "main/test_data", cifar_DHC_out, deflate=3)

cifar_DHC_out_iso = transformMaker(cifar_DHC_out',filter_hash["S1_iso_mat"],filter_hash["S2_iso_mat"],Nc=3)

h5write(filename, "main/test_data_iso", cifar_DHC_out_iso, deflate=3)

Progress: 100%|█████████████████████████████████████████| Time: 0:08:50
Progress: 100%|█████████████████████████████████████████| Time: 0:03:17


In [27]:
train_x_ycbcr = zeros(size(train_x))
for i=1:size(train_x)[4]
    train_x_ycbcr[:,:,:,i] = color_convert(train_x[:,:,:,i],RGB,LMS)
end
test_x_ycbcr = zeros(size(test_x))
for i=1:size(test_x)[4]
    test_x_ycbcr[:,:,:,i] = color_convert(test_x[:,:,:,i],RGB,LMS)
end

lst_train = Array{Any}(undef, 0)
for i = 1:size(train_x_ycbcr)[4]
    push!(lst_train,train_x_ycbcr[:,:,:,i])
end

lst_test = Array{Any}(undef, 0)
for i = 1:size(test_x_ycbcr)[4]
    push!(lst_test,test_x_ycbcr[:,:,:,i])
end

filename = "cifar100_LMS.h5"

h5write(filename, "main/test_labels", test_y, deflate=3)
h5write(filename, "main/train_labels", train_y, deflate=3)

cifar_DHC_out = @showprogress pmap(cifar_DHC, lst_train)
cifar_DHC_out = hcat(cifar_DHC_out...)

h5write(filename, "main/train_data", cifar_DHC_out, deflate=3)

cifar_DHC_out_iso = transformMaker(cifar_DHC_out',filter_hash["S1_iso_mat"],filter_hash["S2_iso_mat"],Nc=3)

h5write(filename, "main/train_data_iso", cifar_DHC_out_iso, deflate=3)

cifar_DHC_out = @showprogress pmap(cifar_DHC, lst_test)
cifar_DHC_out = hcat(cifar_DHC_out...)

h5write(filename, "main/test_data", cifar_DHC_out, deflate=3)

cifar_DHC_out_iso = transformMaker(cifar_DHC_out',filter_hash["S1_iso_mat"],filter_hash["S2_iso_mat"],Nc=3)

h5write(filename, "main/test_data_iso", cifar_DHC_out_iso, deflate=3)

Progress: 100%|█████████████████████████████████████████| Time: 0:08:30
Progress: 100%|█████████████████████████████████████████| Time: 0:03:14


In [28]:
train_x_ycbcr = zeros(size(train_x))
for i=1:size(train_x)[4]
    train_x_ycbcr[:,:,:,i] = color_convert(train_x[:,:,:,i],RGB,DIN99)
end
test_x_ycbcr = zeros(size(test_x))
for i=1:size(test_x)[4]
    test_x_ycbcr[:,:,:,i] = color_convert(test_x[:,:,:,i],RGB,DIN99)
end

lst_train = Array{Any}(undef, 0)
for i = 1:size(train_x_ycbcr)[4]
    push!(lst_train,train_x_ycbcr[:,:,:,i])
end

lst_test = Array{Any}(undef, 0)
for i = 1:size(test_x_ycbcr)[4]
    push!(lst_test,test_x_ycbcr[:,:,:,i])
end

filename = "cifar100_DIN99.h5"

h5write(filename, "main/test_labels", test_y, deflate=3)
h5write(filename, "main/train_labels", train_y, deflate=3)

cifar_DHC_out = @showprogress pmap(cifar_DHC, lst_train)
cifar_DHC_out = hcat(cifar_DHC_out...)

h5write(filename, "main/train_data", cifar_DHC_out, deflate=3)

cifar_DHC_out_iso = transformMaker(cifar_DHC_out',filter_hash["S1_iso_mat"],filter_hash["S2_iso_mat"],Nc=3)

h5write(filename, "main/train_data_iso", cifar_DHC_out_iso, deflate=3)

cifar_DHC_out = @showprogress pmap(cifar_DHC, lst_test)
cifar_DHC_out = hcat(cifar_DHC_out...)

h5write(filename, "main/test_data", cifar_DHC_out, deflate=3)

cifar_DHC_out_iso = transformMaker(cifar_DHC_out',filter_hash["S1_iso_mat"],filter_hash["S2_iso_mat"],Nc=3)

h5write(filename, "main/test_data_iso", cifar_DHC_out_iso, deflate=3)

Progress: 100%|█████████████████████████████████████████| Time: 0:08:46
Progress: 100%|█████████████████████████████████████████| Time: 0:03:19


In [29]:
train_x_ycbcr = zeros(size(train_x))
for i=1:size(train_x)[4]
    train_x_ycbcr[:,:,:,i] = color_convert(train_x[:,:,:,i],RGB,DIN99d)
end
test_x_ycbcr = zeros(size(test_x))
for i=1:size(test_x)[4]
    test_x_ycbcr[:,:,:,i] = color_convert(test_x[:,:,:,i],RGB,DIN99d)
end

lst_train = Array{Any}(undef, 0)
for i = 1:size(train_x_ycbcr)[4]
    push!(lst_train,train_x_ycbcr[:,:,:,i])
end

lst_test = Array{Any}(undef, 0)
for i = 1:size(test_x_ycbcr)[4]
    push!(lst_test,test_x_ycbcr[:,:,:,i])
end

filename = "cifar100_DIN99d.h5"

h5write(filename, "main/test_labels", test_y, deflate=3)
h5write(filename, "main/train_labels", train_y, deflate=3)

cifar_DHC_out = @showprogress pmap(cifar_DHC, lst_train)
cifar_DHC_out = hcat(cifar_DHC_out...)

h5write(filename, "main/train_data", cifar_DHC_out, deflate=3)

cifar_DHC_out_iso = transformMaker(cifar_DHC_out',filter_hash["S1_iso_mat"],filter_hash["S2_iso_mat"],Nc=3)

h5write(filename, "main/train_data_iso", cifar_DHC_out_iso, deflate=3)

cifar_DHC_out = @showprogress pmap(cifar_DHC, lst_test)
cifar_DHC_out = hcat(cifar_DHC_out...)

h5write(filename, "main/test_data", cifar_DHC_out, deflate=3)

cifar_DHC_out_iso = transformMaker(cifar_DHC_out',filter_hash["S1_iso_mat"],filter_hash["S2_iso_mat"],Nc=3)

h5write(filename, "main/test_data_iso", cifar_DHC_out_iso, deflate=3)

Progress: 100%|█████████████████████████████████████████| Time: 0:08:52
Progress: 100%|█████████████████████████████████████████| Time: 0:03:17


In [9]:
train_x_ycbcr = zeros(size(train_x))
for i=1:size(train_x)[4]
    train_x_ycbcr[:,:,:,i] = color_convert(train_x[:,:,:,i],RGB,DIN99o)
end
test_x_ycbcr = zeros(size(test_x))
for i=1:size(test_x)[4]
    test_x_ycbcr[:,:,:,i] = color_convert(test_x[:,:,:,i],RGB,DIN99o)
end

lst_train = Array{Any}(undef, 0)
for i = 1:size(train_x_ycbcr)[4]
    push!(lst_train,train_x_ycbcr[:,:,:,i])
end

lst_test = Array{Any}(undef, 0)
for i = 1:size(test_x_ycbcr)[4]
    push!(lst_test,test_x_ycbcr[:,:,:,i])
end

filename = "cifar100_DIN99o.h5"

h5write(filename, "main/test_labels", test_y, deflate=3)
h5write(filename, "main/train_labels", train_y, deflate=3)

cifar_DHC_out = @showprogress pmap(cifar_DHC, lst_train)
cifar_DHC_out = hcat(cifar_DHC_out...)

h5write(filename, "main/train_data", cifar_DHC_out, deflate=3)

cifar_DHC_out_iso = transformMaker(cifar_DHC_out',filter_hash["S1_iso_mat"],filter_hash["S2_iso_mat"],Nc=3)

h5write(filename, "main/train_data_iso", cifar_DHC_out_iso, deflate=3)

cifar_DHC_out = @showprogress pmap(cifar_DHC, lst_test)
cifar_DHC_out = hcat(cifar_DHC_out...)

h5write(filename, "main/test_data", cifar_DHC_out, deflate=3)

cifar_DHC_out_iso = transformMaker(cifar_DHC_out',filter_hash["S1_iso_mat"],filter_hash["S2_iso_mat"],Nc=3)

h5write(filename, "main/test_data_iso", cifar_DHC_out_iso, deflate=3)

Progress: 100%|█████████████████████████████████████████| Time: 0:09:14
Progress: 100%|█████████████████████████████████████████| Time: 0:01:03
